In [51]:
import gradio as gr
import pandas as pd
import sys
sys.path.append("..")
import torch
from dataset_parsers.MIMIC3 import MIMIC_3
from models import OutcomeModel
from ICDCodesGrouper import ICDCodesGrouper
from utilities import *
from Processing import *
from graph_builders.single import SingleGraph as Graph
import networkx as nx

icdcodes = open('../icd_files/CMS32_DESC_SHORT_DX.txt').read().split('\n')
icdcodes = [x.split(' ')[0] for x in icdcodes]
df = pd.read_csv("../Datasets/MIMIC_3/DIAGNOSES_ICD.csv.gz", compression='gzip')
data = df[df["SUBJECT_ID"] == 23].groupby("HADM_ID")["ICD9_CODE"].apply(lambda x: ','.join(x)).to_frame()
data["VISIT_NUMBER"] = [i for i in range(data.shape[0])]

embedding_size=512
model_path ='../SW-Phenotype-1-512'
model = OutcomeModel(embedding_size=embedding_size,n_layers=1)
model.load_state_dict(torch.load(model_path))
model.eval()

icdgrouper = ICDCodesGrouper(ccs_path='../icd_files/CCS-SingleDiagnosisGrouper.txt')
all_classes = icdgrouper.get_classes('ccs')
patient = lambda x,out: all_classes[out[0].argsort(axis=1,descending=True)[x]]
f_name = "../icd_files/CCS-MultiDiagnosisGrouper.txt"
G= get_or_create_diagnosis_ccs_graph(nx.DiGraph(),f_name,connected=True)
embeddings = init_embeddings(embedding_size,poincare = True,G=G,base_path="../embeddings") 
mimic_path =  "../Datasets/MIMIC_3"
dataset = MIMIC_3(mimic_path)
df = dataset.to_self_supervised(target_grouper = {"phenotype":lambda x: icdgrouper.lookup('ccs',x)})
df = df.loc[["23"]]

def prepare_data(codes):
    # this method only allows to add codes to the next visit.
    # to create several visits minor alterations need to be made.
    new_df = df.copy()
    new_df.diagnoses.iloc[0].append(codes) #add new codes to the first row
    x = new_df.diagnoses.values
    data = get_or_load_data(patients = x,target={},G = G,embedding_size = embedding_size,graph_type=Graph,embeddings=embeddings,replicate=False,random=False)
    return data



def predict(codes):
    history = gr.get_state() or []

    codes = codes.split(',')
    codes = codes + history
    gr.set_state(codes)
    #data = data.append(pd.DataFrame({"VISIT_NUMBER":data.VISIT_NUMBER.max()+1,"ICD9_CODE":codes}))
    #add icd_codes to data..
    inpt = prepare_data(codes)

    res = model(inpt)
    #cross ref with labels and use output label
    label_names = get_names(patient(0,res),icdgrouper.ccs.names)
    scores = res[0].sort(axis=1,descending=True)[0]
    labels = dict(zip(label_names,scores.T.detach().numpy().flatten().tolist()))
    return data,codes, labels



def get_names(sort_tensor,lookup):
    res = []
    for item in sort_tensor:
        res.append(lookup[item])
    return res



iface = gr.Interface(predict,
  [
    gr.inputs.Textbox(lines=7)
  ],
  [
   gr.outputs.Dataframe(label="Patient history"),
   gr.outputs.Textbox(label="Added ICD codes"),
   gr.outputs.Label(label="Next predicted codes",num_top_classes=10)
  ],  
  examples = icdcodes,
  title="Enter ICD codes separated by commas",
  allow_flagging=False,
  allow_screenshot=False,
  article="You selected a patient, and are now wondering where his/her health journey is going. In front of you, you have the entire ICD-9 vocabulary.\nOn your right you can see the patient (real) history, the added ICD-9 codes and the predicted labels. ",
  live=True

)

#iface.test_launch()

iface.launch()


This should never happen.. V173
Running on local URL:  http://127.0.0.1:7869/

To create a public link, set `share=True` in `launch()`.


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7869/', None)